

 <h1>
Welcome to the Math Question Answer Verification Competition! 🚀

The goal is to fine-tune a Llama-3-8B model to predict if a given solution to a math problem is correct or not. Your model should output True if the solution is correct, and False otherwise.

This notebook is a starter guide designed to get you up and running quickly. We'll walk through a simplified training process using a small subset of the data (5,000 examples) and lightweight parameters. The main goal here is to understand the complete workflow, from loading data to generating a submission file, not to achieve a top score.

Good luck, and have fun! 🎉

updated 小仪式

In [2]:
# %%capture
# This hides the (very) long installation output
print("Installing required libraries... This may take 2-3 minutes.")

# --- THIS IS THE ONLY INSTALL COMMAND YOU NEED ---
# It's designed for Colab and will pull in all the correct,
# pre-compiled dependencies, including xformers.
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# --- WE ARE DELETING THE SECOND PIP INSTALL LINE ---
# !pip install --no-deps "xformers<0.0.26" ... (and so on) <-- THIS LINE IS THE PROBLEM. REMOVE IT.

print("Libraries installed successfully!")

# Mount your Google Drive to save the model later
from google.colab import drive
drive.mount('/content/drive')

print("Google Drive mounted at /content/drive")


Installing required libraries... This may take 2-3 minutes.
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-n786mw91/unsloth_7d68463467b2475c922406dc749dabe7
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-n786mw91/unsloth_7d68463467b2475c922406dc749dabe7
  Resolved https://github.com/unslothai/unsloth.git to commit d707bd43b4e883b521761d525be2fae428fe5980
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.6/273.6 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [3]:
print("Fixing version conflicts...")

!pip uninstall -y unsloth unsloth_zoo trl transformers torchao xformers

!pip install --upgrade --force-reinstall --no-cache-dir \
    "unsloth==2025.10.8" \
    "unsloth_zoo==2025.10.9" \
    "trl==0.16.1" \
    "transformers==4.56.1" \
    "accelerate>=1.0.0"

print("✅ Versions installed. NOW restart runtime:")



Fixing version conflicts...
Found existing installation: unsloth 2025.10.12
Uninstalling unsloth-2025.10.12:
  Successfully uninstalled unsloth-2025.10.12
Found existing installation: unsloth_zoo 2025.10.13
Uninstalling unsloth_zoo-2025.10.13:
  Successfully uninstalled unsloth_zoo-2025.10.13
Found existing installation: trl 0.23.0
Uninstalling trl-0.23.0:
  Successfully uninstalled trl-0.23.0
Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: torchao 0.14.1
Uninstalling torchao-0.14.1:
  Successfully uninstalled torchao-0.14.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 194.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 234.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 288.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

✅ Versions installed. NOW restart runtime:


In [1]:
# ✅ Force patch missing RL replacements Hook
import unsloth
from unsloth.models import rl

if "align_logprobs_with_mask" not in rl.RL_REPLACEMENTS:
    print("⚡ Injecting missing RL replacement: align_logprobs_with_mask...")
    rl.RL_REPLACEMENTS["align_logprobs_with_mask"] = lambda *args, **kwargs: None
else:
    print("✅ align_logprobs_with_mask already exists")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
⚡ Injecting missing RL replacement: align_logprobs_with_mask...


In [2]:
import sys, unsloth, transformers, trl
from importlib import import_module

print("Python:", sys.version)
print("unsloth =", unsloth.__version__)
print("transformers =", transformers.__version__)
print("trl =", trl.__version__)

# RL dependency check
rl = import_module("unsloth.models.rl")
print("align_logprobs_with_mask:", "align_logprobs_with_mask" in rl.RL_REPLACEMENTS)


Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
unsloth = 2025.10.8
transformers = 4.56.1
trl = 0.16.1
align_logprobs_with_mask: True


# 1st try. 0.819355 at Oct 29 2025 2AM

In [ ]:
import torch
from unsloth import FastLanguageModel

# --- Configuration ---
max_seq_length = 2048 # Max length for a single problem. 2048 is a good start.
dtype = None          # None auto-detects the best precision (bfloat16)
load_in_4bit = True   # This is a must-have for a free Colab GPU

# This is the specific model from the handout, optimized by Unsloth
model_id = "unsloth/Meta-Llama-3.1-8B"

# Load the model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print("✅ Model and tokenizer loaded.")

==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

✅ Model and tokenizer loaded.


We hit 0.81 for accuracy for this one.

Ask the model to create a solution then compare it to the given solution.

Then give the solution for True or False

In [ ]:
# =========================
# 0) Imports (Unsloth first)
# =========================
import unsloth
from unsloth import FastLanguageModel

import torch
from torch.nn.utils.rnn import pad_sequence
from datasets import load_dataset
from transformers import TrainingArguments, EarlyStoppingCallback
from trl import SFTTrainer

# If you haven't created model+tokenizer earlier in the notebook, uncomment:
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name     = "unsloth/Meta-Llama-3.1-8B",
#     max_seq_length = 2048,
#     dtype          = None,
#     load_in_4bit   = True,
# )

max_seq_length = 2048

# Ensure a pad token and keep the TAIL (label) by truncating from the LEFT
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"   # keep ANSWER_TAG + label at the end

# =========================================
# 1) LoRA (skip if already added by Unsloth)
# =========================================
try:
    model = FastLanguageModel.get_peft_model(
        model,
        r = 8,
        lora_alpha = 16,
        target_modules = [
            "q_proj","k_proj","v_proj","o_proj",
            "gate_proj","up_proj","down_proj",
        ],
        lora_dropout = 0.0,                 # keep Unsloth fast
        bias = "none",
        use_gradient_checkpointing = "unsloth",
        random_state = 42,
    )
    print("✅ LoRA adapters configured.")
except Exception as e:
    print("Unsloth: Already have LoRA adapters! Skipping. (", e, ")")

# =========================================
# 2) Dataset → subset + formatting (label only after cue)
# =========================================
DATASET_ID = "ad6398/nyu-dl-teach-maths-comp"
full_dataset = load_dataset(DATASET_ID, split="train")

train_subset = 10_000
valid_subset = 1_000

split_dataset = full_dataset.train_test_split(
    test_size = valid_subset / (train_subset + valid_subset),
    seed = 42,
)

train_dataset = split_dataset["train"].select(range(train_subset))
validation_dataset = split_dataset["test"].select(range(valid_subset))
print(f"Loaded {len(train_dataset):,} train / {len(validation_dataset):,} validation samples.")

EOS_TOKEN   = tokenizer.eos_token or "<|end_of_text|>"
ANSWER_TAG  = "Your Final Answer (True/False):\n"  # used by the collator mask
RESERVE_TOKENS = 12  # room for ANSWER_TAG + ('True'/'False') + EOS

PROMPT_TEMPLATE = (
    "You are a logical and precise mathematician.\n"
    "First, solve the question step-by-step in your own words.\n"
    "Then, compare your computed answer against the provided solution.\n"
    "Finally, answer with 'True' if the provided solution is correct, otherwise answer 'False'.\n\n"
    "Question:\n{question}\n\n"
    "Provided Solution:\n{provided_solution}\n\n"
    "Your Reasoning:\n(think step-by-step here)\n\n"
    f"{ANSWER_TAG}"
)

def safe_join(question: str, provided_solution: str, label: str) -> str:
    """Keep the tail by trimming the start of long provided_solution if needed."""
    prefix = PROMPT_TEMPLATE.format(question=question, provided_solution=provided_solution)
    ids = tokenizer(prefix, add_special_tokens=False)["input_ids"]
    if len(ids) > max_seq_length - RESERVE_TOKENS:
        extra = len(ids) - (max_seq_length - RESERVE_TOKENS)
        provided_solution = provided_solution[extra:]  # cheap left-trim (works since we truncate-left)
        prefix = PROMPT_TEMPLATE.format(question=question, provided_solution=provided_solution)
    return prefix + label + EOS_TOKEN

def formatting_prompts_func(batch):
    texts = []
    for q, s, o in zip(batch["question"], batch["solution"], batch["is_correct"]):
        label = "True" if bool(o) else "False"
        prompt = safe_join(q.strip(), str(s).strip(), label)
        texts.append(prompt)
    return {"text": texts}

formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True, num_proc=8)
formatted_validation_dataset = validation_dataset.map(formatting_prompts_func, batched=True, num_proc=8)

print("\n✅ Dataset Ready:")
print(f"   {len(formatted_train_dataset)} training samples")
print(f"   {len(formatted_validation_dataset)} validation samples")
print("\n📌 Example Training Sample:\n")
print(formatted_train_dataset[0]["text"][:1000])

# =========================================
# 3) Custom completion-only collator (robust to raw or tokenized features)
# =========================================
class CompletionOnlyCollator:
    def __init__(self, tokenizer, response_template, max_length=2048):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.template_ids = self.tokenizer(
            response_template,
            add_special_tokens=False,
        )["input_ids"]

    def _mask_after_template(self, input_ids: torch.Tensor) -> torch.Tensor:
        tpl = torch.tensor(self.template_ids, dtype=torch.long)
        labels = input_ids.clone()
        start = -1
        # find first occurrence of template ids in sequence
        for i in range(0, max(0, input_ids.size(0) - tpl.size(0) + 1)):
            if torch.equal(input_ids[i:i + tpl.size(0)], tpl):
                start = i
                break
        if start == -1:
            labels[:] = -100
        else:
            cutoff = start + tpl.size(0)
            labels[:cutoff] = -100
        return labels

    def __call__(self, features):
        # Case A: raw text features
        if "text" in features[0]:
            texts = [f["text"] for f in features]
            enc = self.tokenizer(
                texts,
                padding=False,
                truncation=True,
                max_length=self.max_length,
                return_tensors=None,
            )
            input_ids_list = [torch.tensor(x, dtype=torch.long) for x in enc["input_ids"]]
            attn_mask_list = [torch.tensor(x, dtype=torch.long) for x in enc.get("attention_mask", [[1]*len(ids) for ids in enc["input_ids"]])]
        else:
            # Case B: already tokenized (SFTTrainer preprocessed)
            input_ids_list, attn_mask_list = [], []
            for f in features:
                ids = f["input_ids"]
                if isinstance(ids, list):
                    ids = torch.tensor(ids, dtype=torch.long)
                input_ids_list.append(ids)
                if "attention_mask" in f:
                    am = f["attention_mask"]
                    if isinstance(am, list):
                        am = torch.tensor(am, dtype=torch.long)
                    attn_mask_list.append(am)
                else:
                    attn_mask_list.append(torch.ones_like(ids, dtype=torch.long))

        # Build labels by masking everything before ANSWER_TAG
        labels_list = [self._mask_after_template(ids) for ids in input_ids_list]

        # Pad to batch
        pad_id = self.tokenizer.pad_token_id
        input_ids      = pad_sequence(input_ids_list, batch_first=True, padding_value=pad_id)
        attention_mask = pad_sequence(attn_mask_list,  batch_first=True, padding_value=0)
        labels         = pad_sequence(labels_list,     batch_first=True, padding_value=-100)

        # Truncate (safety)
        input_ids      = input_ids[:, :self.max_length]
        attention_mask = attention_mask[:, :self.max_length]
        labels         = labels[:, :self.max_length]

        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

collator = CompletionOnlyCollator(
    tokenizer = tokenizer,
    response_template = ANSWER_TAG,
    max_length = max_seq_length,
)

# =========================================
# 4) Training arguments
# =========================================
output_dir = "/content/drive/MyDrive/llama3_math_contest"

training_args = TrainingArguments(
    output_dir = output_dir,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 8,     # effective batch = 32
    max_steps = 800,                     # bounded runtime
    warmup_steps = 50,
    learning_rate = 2e-4,
    lr_scheduler_type = "linear",

    logging_steps = 25,
    save_strategy = "steps",
    save_steps = 100,
    save_total_limit = 2,

    # Keep best checkpoint: eval & save strategies must match
    eval_strategy = "steps",
    load_best_model_at_end = True,
    metric_for_best_model = "eval_loss",
    greater_is_better = False,

    # IMPORTANT: let our custom collator see all fields
    remove_unused_columns = False,

    bf16 = torch.cuda.is_bf16_supported(),
    fp16 = not torch.cuda.is_bf16_supported(),
    optim = "adamw_8bit",
    weight_decay = 0.02,
    seed = 42,
    report_to = "none",
)

# =========================================
# 5) Trainer (packing=False to preserve the template boundary)
# =========================================
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train_dataset,
    eval_dataset  = formatted_validation_dataset,
    dataset_text_field = "text",   # ok; collator handles both raw or tokenized
    data_collator = collator,
    max_seq_length = max_seq_length,
    packing = False,
    args = training_args,
    eval_steps = 100,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)],
)

print("✅ Trainer initialized.\nStarting training...")
trainer.train()
print("🎉 Training finished!")

# =========================================
# 6) Save
# =========================================
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"✅ Final model saved to: {output_dir}")


Unsloth: Already have LoRA adapters! We shall skip this step.


✅ LoRA adapters configured.
Loaded 10,000 train / 1,000 validation samples.


Map (num_proc=8):   0%|          | 0/10000 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]


✅ Dataset Ready:
   10000 training samples
   1000 validation samples

📌 Example Training Sample:

You are a logical and precise mathematician.
First, solve the question step-by-step in your own words.
Then, compare your computed answer against the provided solution.
Finally, answer with 'True' if the provided solution is correct, otherwise answer 'False'.

Question:
Austin bought his seven friends each a robot. Each robot costs $8.75.  He was charged $7.22 total for tax. He left with $11.53 in change. How much did Austin start with?

Provided Solution:
Let's solve this problem using Python's sympy library.
<llm-code>
import sympy as sp

# initial cost
cost_per_robot = 8.75
cost_of_7_robots = cost_per_robot * 7
# calculate tax
tax_per_robot = cost_per_robot * 7.22 / 100
tax_of_7_robots = tax_per_robot * 7
# subtract tax from cost
final_cost = cost_of_7_robots - tax_of_7_robots
# Austin started with
amount_austin_started_with = final_cost + 11.53
amount_austin_started_with
</llm-code>


...

The model is already on multiple devices. Skipping the move to device specified in `args`.


✅ Trainer initialized.
Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 3 | Total steps = 800
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 20,971,520 of 8,051,232,768 (0.26% trained)


Step,Training Loss,Validation Loss
25,0.246100,0.233075
50,0.270800,0.236260
75,0.234200,0.262186
100,0.246600,0.229187
125,0.226000,0.209086
150,0.224000,0.226401
175,0.217100,0.216431
200,0.243500,0.230509


🎉 Training finished!
✅ Final model saved to: /content/drive/MyDrive/llama3_math_contest


In [ ]:
# 0) LOAD YOUR FINETUNED MODEL
from unsloth import FastLanguageModel
import torch, re, pandas as pd
from tqdm import tqdm
from datasets import load_dataset

# <- set this to where you saved earlier
CKPT_DIR = "/content/drive/MyDrive/llama3_math_contest"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = CKPT_DIR,     # uses your fine-tuned adapters
    max_seq_length = 2048,
    dtype          = None,         # auto: bf16 on A100, else fp16
    load_in_4bit   = True,
)
model.eval().to("cuda")

# 1) INFERENCE PROMPT (same structure as training)
INFERENCE_PROMPT = (
    "You are a logical and precise mathematician.\n"
    "First, solve the question step-by-step in your own words.\n"
    "Then, compare your computed answer against the provided solution.\n"
    "Finally, answer with 'True' if the provided solution is correct, otherwise answer 'False'.\n\n"
    "Question:\n{q}\n\n"
    "Provided Solution:\n{s}\n\n"
    "Your Reasoning:\n(think step-by-step here)\n\n"
    "Your Final Answer (True/False):\n"
)

def parse_true_false(text: str):
    m = re.search(r"\b(True|False)\b", text, flags=re.IGNORECASE)
    if not m:
        return None
    return m.group(1).lower() == "true"

# 2) RUN INFERENCE OVER TEST SET
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

for ex in tqdm(test_dataset):
    q = ex["question"].strip()
    s = str(ex["solution"]).strip()

    prompt = INFERENCE_PROMPT.format(q=q, s=s)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=4,     # just enough to emit True/False
            do_sample=False,      # deterministic
            temperature=0.0
        )

    generated = tokenizer.decode(
        outputs[0][inputs["input_ids"].size(1):],
        skip_special_tokens=True
    )

    pred = parse_true_false(generated)
    if pred is None:
        pred = False  # safe fallback
    predictions.append(pred)

# 3) BUILD SUBMISSION
submission = pd.DataFrame({
    "ID": range(len(predictions)),
    "is_correct": predictions
})
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv saved — ready to upload.")


==((====))==  Unsloth 2025.10.11: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 10000/10000 [53:54<00:00,  3.09it/s]

✅ submission.csv saved — ready to upload.


This is 1st version trying to improve the outcome.

In [ ]:
# Force uninstall and reinstall from Git to get the latest versions
!pip install --upgrade --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install --upgrade --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth-zoo.git


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-zc0bvsaj
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-zc0bvsaj
  Resolved https://github.com/unslothai/unsloth.git to commit 5314c214d21a387791decc6b0f7715ebd7c1eeb7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.10.11-py3-none-any.whl size=351051 sha256=20901d46e191b3313d5cdfb840811487c339b625460f09716666383566427fc3
  Stored in directory: /tmp/pip-ephem-wheel-cache-jbge57i2/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.10.11
    Uninstalling unsloth-2025.10.11:
      Successfully uninstalled unsloth-2025.10.11
  Cloning https://github.com/unslothai/unsloth-zoo.git to /tmp/pip-req-bui

# 7th try. score = 0.84892 submit At Oct 29 2025 10PM
# Started at Oct 29 2025 3PM

In [ ]:
import torch
from unsloth import FastLanguageModel

# --- Configuration ---
max_seq_length = 2048 # Max length for a single problem. 2048 is a good start.
dtype = None          # None auto-detects the best precision (bfloat16)
load_in_4bit = True   # This is a must-have for a free Colab GPU

# This is the specific model from the handout, optimized by Unsloth
model_id = "unsloth/Meta-Llama-3.1-8B"

# Load the model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print("✅ Model and tokenizer loaded.")

==((====))==  Unsloth 2025.10.11: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Model and tokenizer loaded.


In [ ]:
# =========================
# 0) Imports (Unsloth first)
# =========================
import unsloth
from unsloth import FastLanguageModel
from datasets import concatenate_datasets

import torch
from torch.nn.utils.rnn import pad_sequence
from datasets import load_dataset
from transformers import TrainingArguments, EarlyStoppingCallback
from trl import SFTTrainer

max_seq_length = 2048

# Ensure pad + left-truncation
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"

# =========================================
# 1) LoRA
# =========================================
try:
    model = FastLanguageModel.get_peft_model(
        model,
        r = 8,
        lora_alpha = 16,
        target_modules = [
            "q_proj","k_proj","v_proj","o_proj",
            "gate_proj","up_proj","down_proj",
        ],
        lora_dropout = 0.0,
        bias = "none",
        use_gradient_checkpointing = "unsloth",
        random_state = 42,
    )
    print("✅ LoRA adapters configured.")
except:
    print("⚠️ LoRA already present — continuing.")

# =========================================
# 2) Dataset formatting — expand to 60k
# =========================================
DATASET_ID = "ad6398/nyu-dl-teach-maths-comp"
full_dataset = load_dataset(DATASET_ID, split="train").shuffle(seed=42)

# Separate by labels
true_data  = full_dataset.filter(lambda x: x["is_correct"] == True)
false_data = full_dataset.filter(lambda x: x["is_correct"] == False)

print(len(true_data), len(false_data))

# Target size per split
train_per_class = 10_000   # 10k True, 10k False = 20k
val_per_class   = 500      # 500 True, 500 False = 1k

train_true  = true_data.select(range(train_per_class))
train_false = false_data.select(range(train_per_class))
validation_true  = true_data.select(range(train_per_class, train_per_class + val_per_class))
validation_false = false_data.select(range(train_per_class, train_per_class + val_per_class))

# Merge + Shuffle final sets
train_dataset = concatenate_datasets([train_true, train_false]).shuffle(seed=42)
validation_dataset = concatenate_datasets([validation_true, validation_false]).shuffle(seed=42)

print("✅ Final Balanced Dataset:")
print(f"Train size: {len(train_dataset)}   (True={train_per_class}, False={train_per_class})")
print(f"Validation size: {len(validation_dataset)} (True={val_per_class}, False={val_per_class})")

EOS_TOKEN = tokenizer.eos_token or "<|end_of_text|>"
ANSWER_TAG = "Your Final Answer (True/False):\n"
RESERVE_TOKENS = 12

PROMPT_TEMPLATE = (
    "You are a logical and precise mathematician.\n"
    "First, solve the question step-by-step in your own words.\n"
    "Then, compare your computed answer against the provided solution.\n"
    "Finally, answer with 'True' if the provided solution is correct, otherwise answer 'False'.\n\n"
    "Question:\n{question}\n\n"
    "Provided Solution:\n{provided_solution}\n\n"
    "Your Reasoning:\n(think step-by-step here)\n\n"
    f"{ANSWER_TAG}"
)

def safe_join(question, provided_solution, label):
    prefix = PROMPT_TEMPLATE.format(question=question, provided_solution=provided_solution)
    ids = tokenizer(prefix, add_special_tokens=False)["input_ids"]
    if len(ids) > max_seq_length - RESERVE_TOKENS:
        extra = len(ids) - (max_seq_length - RESERVE_TOKENS)
        provided_solution = provided_solution[extra:]
        prefix = PROMPT_TEMPLATE.format(question=question, provided_solution=provided_solution)
    return prefix + label + EOS_TOKEN

def formatting_prompts_func(batch):
    texts = []
    for q, s, o in zip(batch["question"], batch["solution"], batch["is_correct"]):
        label = "True" if bool(o) else "False"
        texts.append(safe_join(q.strip(), str(s).strip(), label))
    return {"text": texts}

formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True, num_proc=8)
formatted_validation_dataset = validation_dataset.map(formatting_prompts_func, batched=True, num_proc=8)

# ✅ Filter out prompts missing answer tag
def _has_answer_tag(example):
    ids = tokenizer(example["text"], add_special_tokens=False)["input_ids"]
    tag_ids = tokenizer(ANSWER_TAG, add_special_tokens=False)["input_ids"]
    L, T = len(ids), len(tag_ids)
    for i in range(L - T + 1):
        if ids[i:i+T] == tag_ids:
            return True
    return False

formatted_train_dataset = formatted_train_dataset.filter(_has_answer_tag, num_proc=8)
formatted_validation_dataset = formatted_validation_dataset.filter(_has_answer_tag, num_proc=8)

print("✅ Final Dataset:", len(formatted_train_dataset), len(formatted_validation_dataset))

# =========================================
# 3) Collator unchanged (good)
# =========================================
class CompletionOnlyCollator:
    def __init__(self, tokenizer, response_template, max_length=2048):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.template_ids = tokenizer(response_template, add_special_tokens=False)["input_ids"]

    def _mask_after_template(self, input_ids):
        tpl = torch.tensor(self.template_ids, dtype=torch.long)
        labels = input_ids.clone()
        start = -1
        L = len(input_ids)
        T = len(tpl)
        for i in range(L - T + 1):
            if torch.equal(input_ids[i:i+T], tpl):
                start = i
                break
        if start == -1:
            labels[:] = -100
        else:
            labels[:start+T] = -100
        return labels

    def __call__(self, features):
        ids_list = []
        attn_list = []
        labels_list = []
        for f in features:
            ids = torch.tensor(self.tokenizer(f["text"], truncation=True, max_length=self.max_length)["input_ids"])
            attn = torch.ones_like(ids)
            labels = self._mask_after_template(ids)
            ids_list.append(ids)
            attn_list.append(attn)
            labels_list.append(labels)
        input_ids = pad_sequence(ids_list, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        attention_mask = pad_sequence(attn_list, batch_first=True, padding_value=0)
        labels = pad_sequence(labels_list, batch_first=True, padding_value=-100)
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

collator = CompletionOnlyCollator(tokenizer, ANSWER_TAG, max_seq_length)

# =========================================
# 4) Training arguments — improved ✅
# =========================================
output_dir = "/content/drive/MyDrive/llama3_math_contest/WEDversion3PM"

training_args = TrainingArguments(
    output_dir = output_dir,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 8,
    max_steps = 1200,
    warmup_steps = 100,
    learning_rate = 5e-5,
    lr_scheduler_type = "cosine",

    logging_steps = 25,
    save_strategy = "steps",
    save_steps = 200,
    save_total_limit = 2,

    eval_strategy = "steps",
    eval_steps = 200,
    load_best_model_at_end = True,
    metric_for_best_model = "eval_loss",
    greater_is_better = False,

    remove_unused_columns = False,

    bf16 = torch.cuda.is_bf16_supported(),
    fp16 = not torch.cuda.is_bf16_supported(),
    optim = "adamw_8bit",
    weight_decay = 0.01,
    seed = 42,
    report_to = "none",
)

# =========================================
# 5) Train
# =========================================
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_train_dataset,
    eval_dataset=formatted_validation_dataset,
    dataset_text_field="text",
    data_collator=collator,
    max_seq_length=max_seq_length,
    packing=False,
    args=training_args,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

print("✅ Begin training!")
trainer.train()
print("🎉 Training finished!")

# =========================================
# 6) Save
# =========================================
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print("✅ Saved model to:", output_dir)


Unsloth: Already have LoRA adapters! We shall skip this step.


✅ LoRA adapters configured.
400000 600000
✅ Final Balanced Dataset:
Train size: 20000   (True=10000, False=10000)
Validation size: 1000 (True=500, False=500)
✅ Final Dataset: 20000 1000


The model is already on multiple devices. Skipping the move to device specified in `args`.


✅ Begin training!


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 2 | Total steps = 1,200
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 20,971,520 of 8,051,232,768 (0.26% trained)


Step,Training Loss,Validation Loss
200,0.217800,0.221511
400,0.205600,0.209472
600,0.184500,0.182037
800,0.160700,0.174698
1000,0.167800,0.169543
1200,0.155900,0.170638


🎉 Training finished!
✅ Saved model to: /content/drive/MyDrive/llama3_math_contest/WEDversion3PM


In [ ]:
# 0) LOAD YOUR FINETUNED MODEL
from unsloth import FastLanguageModel
import torch, re, pandas as pd
from tqdm import tqdm
from datasets import load_dataset

# <- set this to where you saved earlier
CKPT_DIR = "/content/drive/MyDrive/llama3_math_contest/WEDversion3PM"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = CKPT_DIR,     # uses your fine-tuned adapters
    max_seq_length = 2048,
    dtype          = None,         # auto: bf16 on A100, else fp16
    load_in_4bit   = True,
)
model.eval().to("cuda")

# 1) INFERENCE PROMPT (same structure as training)
INFERENCE_PROMPT = (
    "You are a logical and precise mathematician.\n"
    "First, solve the question step-by-step in your own words.\n"
    "Then, compare your computed answer against the provided solution.\n"
    "Finally, answer with 'True' if the provided solution is correct, otherwise answer 'False'.\n\n"
    "Question:\n{q}\n\n"
    "Provided Solution:\n{s}\n\n"
    "Your Reasoning:\n(think step-by-step here)\n\n"
    "Your Final Answer (True/False):\n"
)

def parse_true_false(text: str):
    m = re.search(r"\b(True|False)\b", text, flags=re.IGNORECASE)
    if not m:
        return None
    return m.group(1).lower() == "true"

# 2) RUN INFERENCE OVER TEST SET
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

for ex in tqdm(test_dataset):
    q = ex["question"].strip()
    s = str(ex["solution"]).strip()

    prompt = INFERENCE_PROMPT.format(q=q, s=s)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=4,     # just enough to emit True/False
            do_sample=False,      # deterministic
            temperature=0.0
        )

    generated = tokenizer.decode(
        outputs[0][inputs["input_ids"].size(1):],
        skip_special_tokens=True
    )

    pred = parse_true_false(generated)
    if pred is None:
        pred = False  # safe fallback
    predictions.append(pred)

# 3) BUILD SUBMISSION
submission = pd.DataFrame({
    "ID": range(len(predictions)),
    "is_correct": predictions
})
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv saved — ready to upload.")


==((====))==  Unsloth 2025.10.11: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 10000/10000 [52:03<00:00,  3.20it/s]

✅ submission.csv saved — ready to upload.


# This is 8th submission. start at OCT 29 2025.10:57PM

Using larger train 20k. Trying to increase the score.

50/50 true false question.




In [ ]:
# =========================
# 0) Imports (Unsloth first)
# =========================
import unsloth
from unsloth import FastLanguageModel
from datasets import load_dataset, concatenate_datasets

import torch
from torch.nn.utils.rnn import pad_sequence
from transformers import TrainingArguments, EarlyStoppingCallback
from trl import SFTTrainer

import pandas as pd
from tqdm import tqdm

# --- Base config ---
max_seq_length = 2048
dtype = None          # auto bf16 on A100 / fp16 otherwise
load_in_4bit = True   # good for L4/A100

# --- Load base model once (for finetuning) ---
model_id = "unsloth/Meta-Llama-3.1-8B"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = model_id,
    max_seq_length = max_seq_length,
    dtype          = dtype,
    load_in_4bit   = load_in_4bit,
)
print("✅ Model and tokenizer loaded for training.")

# Ensure pad + left-truncation + left-padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"
tokenizer.padding_side    = "left"  # optional but consistent

# =========================================
# 1) LoRA
# =========================================
try:
    model = FastLanguageModel.get_peft_model(
        model,
        r = 8,
        lora_alpha = 16,
        target_modules = [
            "q_proj","k_proj","v_proj","o_proj",
            "gate_proj","up_proj","down_proj",
        ],
        lora_dropout = 0.0,
        bias = "none",
        use_gradient_checkpointing = "unsloth",
        random_state = 42,
    )
    print("✅ LoRA adapters configured.")
except Exception:
    print("⚠️ LoRA already present — continuing.")

# =========================================
# 2) Dataset formatting — balanced 20k train / 1k val
# =========================================
DATASET_ID = "ad6398/nyu-dl-teach-maths-comp"
full_dataset = load_dataset(DATASET_ID, split="train").shuffle(seed=42)

# Split by label (after global shuffle)
true_data  = full_dataset.filter(lambda x: x["is_correct"] == True)
false_data = full_dataset.filter(lambda x: x["is_correct"] == False)
print(len(true_data), len(false_data))

# Target sizes
train_per_class = 10_000   # 10k True + 10k False = 20k
val_per_class   = 500      # 0.5k True + 0.5k False = 1k

train_true         = true_data.select(range(train_per_class))
train_false        = false_data.select(range(train_per_class))
validation_true    = true_data.select(range(train_per_class, train_per_class + val_per_class))
validation_false   = false_data.select(range(train_per_class, train_per_class + val_per_class))

# Merge + shuffle final sets
from datasets import concatenate_datasets
train_dataset       = concatenate_datasets([train_true, train_false]).shuffle(seed=42)
validation_dataset  = concatenate_datasets([validation_true, validation_false]).shuffle(seed=42)

print("✅ Final Balanced Dataset:")
print(f"Train size: {len(train_dataset)}   (True={train_per_class}, False={train_per_class})")
print(f"Validation size: {len(validation_dataset)} (True={val_per_class}, False={val_per_class})")

EOS_TOKEN      = tokenizer.eos_token or "<|end_of_text|>"
ANSWER_TAG     = "Your Final Answer (True/False):\n"
RESERVE_TOKENS = 12

PROMPT_TEMPLATE = (
    "You are a strict and highly accurate math verifier.\n"
    "Your task:\n"
    "1) Recalculate the correct answer yourself.\n"
    "2) Compare it with the provided solution ONLY.\n"
    "3) Judge if the provided solution is correct.\n"
    "Important rules:\n"
    "- Think privately inside <scratch></scratch>.\n"
    "- DO NOT reveal intermediate calculations.\n"
    "- Output ONLY 'True' or 'False' after the final answer tag.\n\n"
    "Question:\n{question}\n\n"
    "Provided Solution:\n{provided_solution}\n\n"
    "<scratch>Step-by-step internal reasoning here (not visible to users)</scratch>\n"
    f"{ANSWER_TAG}"
)

def safe_join(question: str, provided_solution: str, label: str) -> str:
    prefix = PROMPT_TEMPLATE.format(question=question, provided_solution=provided_solution)
    ids = tokenizer(prefix, add_special_tokens=False)["input_ids"]
    if len(ids) > max_seq_length - RESERVE_TOKENS:
        extra = len(ids) - (max_seq_length - RESERVE_TOKENS)
        provided_solution = provided_solution[extra:]
        prefix = PROMPT_TEMPLATE.format(question=question, provided_solution=provided_solution)
    return prefix + label + EOS_TOKEN

def formatting_prompts_func(batch):
    texts = []
    for q, s, o in zip(batch["question"], batch["solution"], batch["is_correct"]):
        label = "True" if bool(o) else "False"
        texts.append(safe_join(q.strip(), str(s).strip(), label))
    return {"text": texts}

formatted_train_dataset      = train_dataset.map(formatting_prompts_func,      batched=True, num_proc=8)
formatted_validation_dataset = validation_dataset.map(formatting_prompts_func, batched=True, num_proc=8)

# Ensure the ANSWER_TAG exists (safety)
def _has_answer_tag(example):
    ids = tokenizer(example["text"], add_special_tokens=False)["input_ids"]
    tag_ids = tokenizer(ANSWER_TAG, add_special_tokens=False)["input_ids"]
    L, T = len(ids), len(tag_ids)
    for i in range(max(0, L - T + 1)):
        if ids[i:i+T] == tag_ids:
            return True
    return False

formatted_train_dataset      = formatted_train_dataset.filter(_has_answer_tag,      num_proc=8)
formatted_validation_dataset = formatted_validation_dataset.filter(_has_answer_tag, num_proc=8)

print("✅ Final Dataset:", len(formatted_train_dataset), len(formatted_validation_dataset))

# =========================================
# 3) Collator — completion-only after ANSWER_TAG
# =========================================
class CompletionOnlyCollator:
    def __init__(self, tokenizer, response_template, max_length=2048):
        self.tokenizer     = tokenizer
        self.max_length    = max_length
        self.template_ids  = tokenizer(response_template, add_special_tokens=False)["input_ids"]

    def _mask_after_template(self, input_ids: torch.Tensor) -> torch.Tensor:
        tpl    = torch.tensor(self.template_ids, dtype=torch.long)
        labels = input_ids.clone()
        L, T   = len(input_ids), len(tpl)
        start  = -1
        for i in range(max(0, L - T + 1)):
            if torch.equal(input_ids[i:i+T], tpl):
                start = i
                break
        if start == -1:
            labels[:] = -100
        else:
            labels[:start+T] = -100
        return labels

    def __call__(self, features):
        ids_list, attn_list, labels_list = [], [], []
        for f in features:
            enc   = self.tokenizer(f["text"], truncation=True, max_length=self.max_length, add_special_tokens=False)
            ids   = torch.tensor(enc["input_ids"], dtype=torch.long)
            attn  = torch.ones_like(ids, dtype=torch.long)
            labs  = self._mask_after_template(ids)
            ids_list.append(ids); attn_list.append(attn); labels_list.append(labs)
        input_ids      = pad_sequence(ids_list,  batch_first=True, padding_value=self.tokenizer.pad_token_id)
        attention_mask = pad_sequence(attn_list, batch_first=True, padding_value=0)
        labels         = pad_sequence(labels_list, batch_first=True, padding_value=-100)
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

collator = CompletionOnlyCollator(tokenizer, ANSWER_TAG, max_seq_length)

# =========================================
# 4) Training args
# =========================================
output_dir = "/content/drive/MyDrive/llama3_math_contest/WEDversion10PM"

training_args = TrainingArguments(
    output_dir = output_dir,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 8,
    max_steps = 1800,
    warmup_steps = 100,
    learning_rate = 8e-5,
    lr_scheduler_type = "cosine",

    logging_steps     = 25,
    save_strategy     = "steps",
    save_steps        = 200,
    save_total_limit  = 2,

    eval_strategy           = "steps",
    eval_steps              = 200,
    load_best_model_at_end  = True,
    metric_for_best_model   = "eval_loss",
    greater_is_better       = False,

    remove_unused_columns = False,

    bf16  = torch.cuda.is_bf16_supported(),
    fp16  = not torch.cuda.is_bf16_supported(),
    optim = "adamw_8bit",
    weight_decay = 0.01,
    seed = 42,
    report_to = "none",
)

# =========================================
# 5) Train
# =========================================
trainer = SFTTrainer(
    model              = model,
    tokenizer          = tokenizer,
    train_dataset      = formatted_train_dataset,
    eval_dataset       = formatted_validation_dataset,
    dataset_text_field = "text",
    data_collator      = collator,
    max_seq_length     = max_seq_length,
    packing            = False,
    args               = training_args,
    callbacks          = [EarlyStoppingCallback(early_stopping_patience=3)],
)

print("✅ Begin training!")
trainer.train()
print("🎉 Training finished!")

# =========================================
# 6) Save
# =========================================
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print("✅ Saved model to:", output_dir)

# =========================================
# 7) Inference + submission
# =========================================
# Reload best checkpoint adapters
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = output_dir,
    max_seq_length = 2048,
    dtype          = None,
    load_in_4bit   = True,
)
model.eval().to("cuda")

def parse_true_false(text: str) -> bool:
    # robust to punctuation/newlines/spaces
    t = text.strip().lower()
    if t.startswith("true"):
        return True
    if t.startswith("false"):
        return False
    # fallback: first occurrence
    if "true" in t and "false" not in t:  return True
    if "false" in t and "true"  not in t: return False
    return False

test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

for ex in tqdm(test_dataset):
    q = ex["question"].strip()
    s = str(ex["solution"]).strip()

    prompt = PROMPT_TEMPLATE.format(question=q, provided_solution=s)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens = 8,     # small headroom
            do_sample      = False,
            temperature    = 0.0,
            eos_token_id   = tokenizer.eos_token_id,
            use_cache      = True,
        )

    # decode ONLY newly generated tokens (safer than splitting the full text)
    new_tokens = outputs[0, inputs["input_ids"].size(1):]
    decoded    = tokenizer.decode(new_tokens, skip_special_tokens=True)

    pred = parse_true_false(decoded)
    predictions.append(pred)

submission = pd.DataFrame({
    "ID": range(len(predictions)),
    "is_correct": predictions
})
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv saved — upload to Kaggle!")


==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Model and tokenizer loaded for training.
✅ LoRA adapters configured.
400000 600000
✅ Final Balanced Dataset:
Train size: 20000   (True=10000, False=10000)
Validation size: 1000 (True=500, False=500)


Map (num_proc=8):   0%|          | 0/20000 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter (num_proc=8):   0%|          | 0/20000 [00:00<?, ? examples/s]

Filter (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]

...

Step,Training Loss,Validation Loss
200,0.205800,0.203653
400,0.199000,0.193409
600,0.183900,0.191887
800,0.141800,0.156165
1000,0.158300,0.178278
1200,0.131300,0.151741
1400,0.064200,0.190124


Step,Training Loss,Validation Loss
200,0.205800,0.203653
400,0.199000,0.193409
600,0.183900,0.191887
800,0.141800,0.156165
1000,0.158300,0.178278
1200,0.131300,0.151741
1400,0.064200,0.190124
1600,0.054200,0.205172
1800,0.043600,0.208331


🎉 Training finished!
✅ Saved model to: /content/drive/MyDrive/llama3_math_contest/WEDversion10PM
==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 10000/10000 [54:35<00:00,  3.05it/s]

✅ submission.csv saved — upload to Kaggle!


# This is 9th submission
here apply dropout to see if the validation loss can decrease.
all other settings stays the same.
This one get 0.87608 for final score


In [ ]:
# =========================
# 0) Imports (Unsloth first)
# =========================
import unsloth
from unsloth import FastLanguageModel
from datasets import load_dataset, concatenate_datasets

import torch
from torch.nn.utils.rnn import pad_sequence
from transformers import TrainingArguments, EarlyStoppingCallback
from trl import SFTTrainer

import pandas as pd
from tqdm import tqdm

# --- Base config ---
max_seq_length = 2048
dtype = None          # auto bf16 on A100 / fp16 otherwise
load_in_4bit = True   # good for L4/A100

# --- Load base model once (for finetuning) ---
model_id = "unsloth/Meta-Llama-3.1-8B"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = model_id,
    max_seq_length = max_seq_length,
    dtype          = dtype,
    load_in_4bit   = load_in_4bit,
)
print("✅ Model and tokenizer loaded for training.")

# Ensure pad + left-truncation + left-padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"
tokenizer.padding_side    = "left"  # optional but consistent

# =========================================
# 1) LoRA
# =========================================
try:
    model = FastLanguageModel.get_peft_model(
        model,
        r = 16,
        lora_alpha = 32,
        target_modules = [
            "q_proj","k_proj","v_proj","o_proj",
            "gate_proj","up_proj","down_proj",
        ],
        lora_dropout = 0.05,
        bias = "none",
        use_gradient_checkpointing = "unsloth",
        random_state = 42,
    )
    print("✅ LoRA adapters configured.")
except Exception:
    print("⚠️ LoRA already present — continuing.")

# =========================================
# 2) Dataset formatting — balanced 20k train / 1k val
# =========================================
DATASET_ID = "ad6398/nyu-dl-teach-maths-comp"
full_dataset = load_dataset(DATASET_ID, split="train").shuffle(seed=42)

# Split by label (after global shuffle)
true_data  = full_dataset.filter(lambda x: x["is_correct"] == True)
false_data = full_dataset.filter(lambda x: x["is_correct"] == False)
print(len(true_data), len(false_data))

# Target sizes
train_per_class = 45_500   # 10k True + 10k False = 20k
val_per_class   = 1000      # 0.5k True + 0.5k False = 1k

train_true         = true_data.select(range(train_per_class))
train_false        = false_data.select(range(train_per_class))
validation_true    = true_data.select(range(train_per_class, train_per_class + val_per_class))
validation_false   = false_data.select(range(train_per_class, train_per_class + val_per_class))

# Merge + shuffle final sets
from datasets import concatenate_datasets
train_dataset       = concatenate_datasets([train_true, train_false]).shuffle(seed=42)
validation_dataset  = concatenate_datasets([validation_true, validation_false]).shuffle(seed=42)

print("✅ Final Balanced Dataset:")
print(f"Train size: {len(train_dataset)}   (True={train_per_class}, False={train_per_class})")
print(f"Validation size: {len(validation_dataset)} (True={val_per_class}, False={val_per_class})")

EOS_TOKEN      = tokenizer.eos_token or "<|end_of_text|>"
ANSWER_TAG     = "Your Final Answer (True/False):\n"
RESERVE_TOKENS = 12

PROMPT_TEMPLATE = (
    "You are a strict and highly accurate math verifier. And you are doing your best. Because you know you are getting 50$ if you made a right verification\n"
    "Your task:\n"
    "1) Recalculate the correct answer yourself. \n"
    "2) Compare it with the provided solution ONLY.\n"
    "3) Judge if the provided solution is correct.\n"
    "4) Step by step, make sure to check provided solution step by step.\n"
    "5) Becarful to the small things!\n"
    "Important rules:\n"
    "- Think privately inside <scratch></scratch>.\n"
    "- DO NOT reveal intermediate calculations.\n"
    "- Output ONLY 'True' or 'False' after the final answer tag.\n\n"
    "Question:\n{question}\n\n"
    "<scratch>Step-by-step internal reasoning here (not visible to users)</scratch>\n"
    "Provided Solution:\n{provided_solution}\n\n"
    f"{ANSWER_TAG}"
)

def safe_join(question: str, provided_solution: str, label: str) -> str:
    prefix = PROMPT_TEMPLATE.format(question=question, provided_solution=provided_solution)
    ids = tokenizer(prefix, add_special_tokens=False)["input_ids"]
    if len(ids) > max_seq_length - RESERVE_TOKENS:
        extra = len(ids) - (max_seq_length - RESERVE_TOKENS)
        provided_solution = provided_solution[extra:]
        prefix = PROMPT_TEMPLATE.format(question=question, provided_solution=provided_solution)
    return prefix + label + EOS_TOKEN

def formatting_prompts_func(batch):
    texts = []
    for q, s, o in zip(batch["question"], batch["solution"], batch["is_correct"]):
        label = "True" if bool(o) else "False"
        texts.append(safe_join(q.strip(), str(s).strip(), label))
    return {"text": texts}

formatted_train_dataset      = train_dataset.map(formatting_prompts_func,      batched=True, num_proc=8)
formatted_validation_dataset = validation_dataset.map(formatting_prompts_func, batched=True, num_proc=8)

# Ensure the ANSWER_TAG exists (safety)
def _has_answer_tag(example):
    ids = tokenizer(example["text"], add_special_tokens=False)["input_ids"]
    tag_ids = tokenizer(ANSWER_TAG, add_special_tokens=False)["input_ids"]
    L, T = len(ids), len(tag_ids)
    for i in range(max(0, L - T + 1)):
        if ids[i:i+T] == tag_ids:
            return True
    return False

formatted_train_dataset      = formatted_train_dataset.filter(_has_answer_tag,      num_proc=8)
formatted_validation_dataset = formatted_validation_dataset.filter(_has_answer_tag, num_proc=8)

print("✅ Final Dataset:", len(formatted_train_dataset), len(formatted_validation_dataset))

# =========================================
# 3) Collator — completion-only after ANSWER_TAG
# =========================================
class CompletionOnlyCollator:
    def __init__(self, tokenizer, response_template, max_length=2048):
        self.tokenizer     = tokenizer
        self.max_length    = max_length
        self.template_ids  = tokenizer(response_template, add_special_tokens=False)["input_ids"]

    def _mask_after_template(self, input_ids: torch.Tensor) -> torch.Tensor:
        tpl    = torch.tensor(self.template_ids, dtype=torch.long)
        labels = input_ids.clone()
        L, T   = len(input_ids), len(tpl)
        start  = -1
        for i in range(max(0, L - T + 1)):
            if torch.equal(input_ids[i:i+T], tpl):
                start = i
                break
        if start == -1:
            labels[:] = -100
        else:
            labels[:start+T] = -100
        return labels

    def __call__(self, features):
        ids_list, attn_list, labels_list = [], [], []
        for f in features:
            enc   = self.tokenizer(f["text"], truncation=True, max_length=self.max_length, add_special_tokens=False)
            ids   = torch.tensor(enc["input_ids"], dtype=torch.long)
            attn  = torch.ones_like(ids, dtype=torch.long)
            labs  = self._mask_after_template(ids)
            ids_list.append(ids); attn_list.append(attn); labels_list.append(labs)
        input_ids      = pad_sequence(ids_list,  batch_first=True, padding_value=self.tokenizer.pad_token_id)
        attention_mask = pad_sequence(attn_list, batch_first=True, padding_value=0)
        labels         = pad_sequence(labels_list, batch_first=True, padding_value=-100)
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

collator = CompletionOnlyCollator(tokenizer, ANSWER_TAG, max_seq_length)

# =========================================
# 4) Training args
# =========================================
output_dir = "/content/drive/MyDrive/llama3_math_contest/Thurversion9PM"

training_args = TrainingArguments(
    output_dir = output_dir,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 8,
    max_steps = 1500,
    warmup_steps = 100,
    learning_rate = 6e-5,
    lr_scheduler_type = "cosine",

    logging_steps     = 25,
    save_strategy     = "steps",
    save_steps        = 150,
    save_total_limit  = 2,

    eval_strategy           = "steps",
    eval_steps              = 150,
    load_best_model_at_end  = True,
    metric_for_best_model   = "eval_loss",
    greater_is_better       = False,

    remove_unused_columns = False,

    bf16  = torch.cuda.is_bf16_supported(),
    fp16  = not torch.cuda.is_bf16_supported(),
    optim = "adamw_8bit",
    weight_decay = 0.02,
    seed = 42,
    report_to = "none",
)

# =========================================
# 5) Train
# =========================================
trainer = SFTTrainer(
    model              = model,
    tokenizer          = tokenizer,
    train_dataset      = formatted_train_dataset,
    eval_dataset       = formatted_validation_dataset,
    dataset_text_field = "text",
    data_collator      = collator,
    max_seq_length     = max_seq_length,
    packing            = False,
    args               = training_args,
    callbacks          = [EarlyStoppingCallback(early_stopping_patience=3)],
)

print("✅ Begin training!")
trainer.train()
print("🎉 Training finished!")

# =========================================
# 6) Save
# =========================================
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print("✅ Saved model to:", output_dir)

# =========================================
# 7) Inference + submission
# =========================================
# Reload best checkpoint adapters
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = output_dir,
    max_seq_length = 2048,
    dtype          = None,
    load_in_4bit   = True,
)
model.eval().to("cuda")

def parse_true_false(text: str) -> bool:
    # robust to punctuation/newlines/spaces
    t = text.strip().lower()
    if t.startswith("true"):
        return True
    if t.startswith("false"):
        return False
    # fallback: first occurrence
    if "true" in t and "false" not in t:  return True
    if "false" in t and "true"  not in t: return False
    return False

test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

for ex in tqdm(test_dataset):
    q = ex["question"].strip()
    s = str(ex["solution"]).strip()

    prompt = PROMPT_TEMPLATE.format(question=q, provided_solution=s)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens = 8,     # small headroom
            do_sample      = False,
            temperature    = 0.0,
            eos_token_id   = tokenizer.eos_token_id,
            use_cache      = True,
        )

    # decode ONLY newly generated tokens (safer than splitting the full text)
    new_tokens = outputs[0, inputs["input_ids"].size(1):]
    decoded    = tokenizer.decode(new_tokens, skip_special_tokens=True)

    pred = parse_true_false(decoded)
    predictions.append(pred)

submission = pd.DataFrame({
    "ID": range(len(predictions)),
    "is_correct": predictions
})
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv saved — upload to Kaggle!")


==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

...

Step,Training Loss,Validation Loss
150,0.237000,0.245982
300,0.200600,0.187206
450,0.183400,0.195370
600,0.179800,0.167215
750,0.147800,0.155912
900,0.139900,0.150780
1050,0.132200,0.143113
1200,0.138300,0.144130
1350,0.131200,0.137388
1500,0.127600,0.137381


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


🎉 Training finished!
✅ Saved model to: /content/drive/MyDrive/llama3_math_contest/Thurversion9PM
==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 10000/10000 [45:14<00:00,  3.68it/s]

✅ submission.csv saved — upload to Kaggle!


# This is 10th submission
I will update the 9th submission
Increase the steps because validation loss is still dropping.
Ans try some small changes in training
Weight decay increase a little.

Increase the prize from 50 to 200.

0.88996
for the final score.

In [ ]:
# =========================
# 0) Imports (Unsloth first)
# =========================
import unsloth
from unsloth import FastLanguageModel
from datasets import load_dataset, concatenate_datasets

import torch
from torch.nn.utils.rnn import pad_sequence
from transformers import TrainingArguments, EarlyStoppingCallback
from trl import SFTTrainer

import pandas as pd
from tqdm import tqdm

# --- Base config ---
max_seq_length = 2048
dtype = None          # auto bf16 on A100 / fp16 otherwise
load_in_4bit = True   # good for L4/A100

# --- Load base model once (for finetuning) ---
model_id = "unsloth/Meta-Llama-3.1-8B"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = model_id,
    max_seq_length = max_seq_length,
    dtype          = dtype,
    load_in_4bit   = load_in_4bit,
)
print("✅ Model and tokenizer loaded for training.")

# Ensure pad + left-truncation + left-padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"
tokenizer.padding_side    = "left"  # optional but consistent

# =========================================
# 1) LoRA
# =========================================
try:
    model = FastLanguageModel.get_peft_model(
        model,
        r = 16,
        lora_alpha = 32,
        target_modules = [
            "q_proj","k_proj","v_proj","o_proj",
            "gate_proj","up_proj","down_proj",
        ],
        lora_dropout = 0.05,
        bias = "none",
        use_gradient_checkpointing = "unsloth",
        random_state = 42,
    )
    print("✅ LoRA adapters configured.")
except Exception:
    print("⚠️ LoRA already present — continuing.")

# =========================================
# 2) Dataset formatting — balanced 20k train / 1k val
# =========================================
DATASET_ID = "ad6398/nyu-dl-teach-maths-comp"
full_dataset = load_dataset(DATASET_ID, split="train").shuffle(seed=42)

# Split by label (after global shuffle)
true_data  = full_dataset.filter(lambda x: x["is_correct"] == True)
false_data = full_dataset.filter(lambda x: x["is_correct"] == False)
print(len(true_data), len(false_data))

# Target sizes
train_per_class = 45_000   # 10k True + 10k False = 20k
val_per_class   = 2000      # 0.5k True + 0.5k False = 1k

train_true         = true_data.select(range(train_per_class))
train_false        = false_data.select(range(train_per_class))
validation_true    = true_data.select(range(train_per_class, train_per_class + val_per_class))
validation_false   = false_data.select(range(train_per_class, train_per_class + val_per_class))

# Merge + shuffle final sets
from datasets import concatenate_datasets
train_dataset       = concatenate_datasets([train_true, train_false]).shuffle(seed=42)
validation_dataset  = concatenate_datasets([validation_true, validation_false]).shuffle(seed=42)

print("✅ Final Balanced Dataset:")
print(f"Train size: {len(train_dataset)}   (True={train_per_class}, False={train_per_class})")
print(f"Validation size: {len(validation_dataset)} (True={val_per_class}, False={val_per_class})")

EOS_TOKEN      = tokenizer.eos_token or "<|end_of_text|>"
ANSWER_TAG     = "Your Final Answer (True/False):\n"
RESERVE_TOKENS = 12

PROMPT_TEMPLATE = (
    "You are a strict and highly accurate math verifier. And you are doing your best. Because you know you are getting $200 if you made a right verification\n You will loss $300 if you made a wrong verication!\n"
    "Your task:\n"
    "1) Recalculate the correct answer yourself.Make sure you consider everything so the solution and the answer you got it right! \n"
    "2) Compare it with the provided solution ONLY.\n"
    "3) Judge if the provided solution is correct.\n"
    "4) Step by step, make sure to check provided solution step by step.\n"
    "5) Becarful to the all small things!\n"
    "Important rules:\n"
    "- Think privately inside <scratch></scratch>.\n"
    "- DO NOT reveal intermediate calculations.\n"
    "- Output ONLY 'True' or 'False' after the final answer tag.\n\n"
    "Question:\n{question}\n\n"
    "<scratch>Step-by-step internal reasoning here (not visible to users)</scratch>\n"
    "Provided Solution:\n{provided_solution}\n\n"
    f"{ANSWER_TAG}"
)

def safe_join(question: str, provided_solution: str, label: str) -> str:
    prefix = PROMPT_TEMPLATE.format(question=question, provided_solution=provided_solution)
    ids = tokenizer(prefix, add_special_tokens=False)["input_ids"]
    if len(ids) > max_seq_length - RESERVE_TOKENS:
        extra = len(ids) - (max_seq_length - RESERVE_TOKENS)
        provided_solution = provided_solution[extra:]
        prefix = PROMPT_TEMPLATE.format(question=question, provided_solution=provided_solution)
    return prefix + label + EOS_TOKEN

def formatting_prompts_func(batch):
    texts = []
    for q, s, o in zip(batch["question"], batch["solution"], batch["is_correct"]):
        label = "True" if bool(o) else "False"
        texts.append(safe_join(q.strip(), str(s).strip(), label))
    return {"text": texts}

formatted_train_dataset      = train_dataset.map(formatting_prompts_func,      batched=True, num_proc=8)
formatted_validation_dataset = validation_dataset.map(formatting_prompts_func, batched=True, num_proc=8)

# Ensure the ANSWER_TAG exists (safety)
def _has_answer_tag(example):
    ids = tokenizer(example["text"], add_special_tokens=False)["input_ids"]
    tag_ids = tokenizer(ANSWER_TAG, add_special_tokens=False)["input_ids"]
    L, T = len(ids), len(tag_ids)
    for i in range(max(0, L - T + 1)):
        if ids[i:i+T] == tag_ids:
            return True
    return False

formatted_train_dataset      = formatted_train_dataset.filter(_has_answer_tag,      num_proc=8)
formatted_validation_dataset = formatted_validation_dataset.filter(_has_answer_tag, num_proc=8)

print("✅ Final Dataset:", len(formatted_train_dataset), len(formatted_validation_dataset))

# =========================================
# 3) Collator — completion-only after ANSWER_TAG
# =========================================
class CompletionOnlyCollator:
    def __init__(self, tokenizer, response_template, max_length=2048):
        self.tokenizer     = tokenizer
        self.max_length    = max_length
        self.template_ids  = tokenizer(response_template, add_special_tokens=False)["input_ids"]

    def _mask_after_template(self, input_ids: torch.Tensor) -> torch.Tensor:
        tpl    = torch.tensor(self.template_ids, dtype=torch.long)
        labels = input_ids.clone()
        L, T   = len(input_ids), len(tpl)
        start  = -1
        for i in range(max(0, L - T + 1)):
            if torch.equal(input_ids[i:i+T], tpl):
                start = i
                break
        if start == -1:
            labels[:] = -100
        else:
            labels[:start+T] = -100
        return labels

    def __call__(self, features):
        ids_list, attn_list, labels_list = [], [], []
        for f in features:
            enc   = self.tokenizer(f["text"], truncation=True, max_length=self.max_length, add_special_tokens=False)
            ids   = torch.tensor(enc["input_ids"], dtype=torch.long)
            attn  = torch.ones_like(ids, dtype=torch.long)
            labs  = self._mask_after_template(ids)
            ids_list.append(ids); attn_list.append(attn); labels_list.append(labs)
        input_ids      = pad_sequence(ids_list,  batch_first=True, padding_value=self.tokenizer.pad_token_id)
        attention_mask = pad_sequence(attn_list, batch_first=True, padding_value=0)
        labels         = pad_sequence(labels_list, batch_first=True, padding_value=-100)
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

collator = CompletionOnlyCollator(tokenizer, ANSWER_TAG, max_seq_length)

# =========================================
# 4) Training args
# =========================================
output_dir = "/content/drive/MyDrive/llama3_math_contest/SATversion1AM"

training_args = TrainingArguments(
    output_dir = output_dir,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 8,
    max_steps = 3000,
    warmup_steps = 150,
    learning_rate = 5e-5,
    lr_scheduler_type = "cosine",

    logging_steps     = 25,
    save_strategy     = "steps",
    save_steps        = 150,
    save_total_limit  = 2,

    eval_strategy           = "steps",
    eval_steps              = 150,
    load_best_model_at_end  = True,
    metric_for_best_model   = "eval_loss",
    greater_is_better       = False,

    remove_unused_columns = False,

    bf16  = torch.cuda.is_bf16_supported(),
    fp16  = not torch.cuda.is_bf16_supported(),
    optim = "adamw_8bit",
    weight_decay = 0.03,
    seed = 42,
    report_to = "none",
)

# =========================================
# 5) Train
# =========================================
trainer = SFTTrainer(
    model              = model,
    tokenizer          = tokenizer,
    train_dataset      = formatted_train_dataset,
    eval_dataset       = formatted_validation_dataset,
    dataset_text_field = "text",
    data_collator      = collator,
    max_seq_length     = max_seq_length,
    packing            = False,
    args               = training_args,
    callbacks          = [EarlyStoppingCallback(early_stopping_patience=3)],
)

print("✅ Begin training!")
trainer.train()
print("🎉 Training finished!")

# =========================================
# 6) Save
# =========================================
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print("✅ Saved model to:", output_dir)

# =========================================
# 7) Inference + submission
# =========================================
# Reload best checkpoint adapters
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = output_dir,
    max_seq_length = 2048,
    dtype          = None,
    load_in_4bit   = True,
)
model.eval().to("cuda")

def parse_true_false(text: str) -> bool:
    # robust to punctuation/newlines/spaces
    t = text.strip().lower()
    if t.startswith("true"):
        return True
    if t.startswith("false"):
        return False
    # fallback: first occurrence
    if "true" in t and "false" not in t:  return True
    if "false" in t and "true"  not in t: return False
    return False

test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

for ex in tqdm(test_dataset):
    q = ex["question"].strip()
    s = str(ex["solution"]).strip()

    prompt = PROMPT_TEMPLATE.format(question=q, provided_solution=s)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens = 8,     # small headroom
            do_sample      = False,
            temperature    = 0.0,
            eos_token_id   = tokenizer.eos_token_id,
            use_cache      = True,
        )

    # decode ONLY newly generated tokens (safer than splitting the full text)
    new_tokens = outputs[0, inputs["input_ids"].size(1):]
    decoded    = tokenizer.decode(new_tokens, skip_special_tokens=True)

    pred = parse_true_false(decoded)
    predictions.append(pred)

submission = pd.DataFrame({
    "ID": range(len(predictions)),
    "is_correct": predictions
})
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv saved — upload to Kaggle!")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


✅ Model and tokenizer loaded for training.


Unsloth 2025.10.8 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


...

Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


Step,Training Loss,Validation Loss
150,0.261100,0.246532
300,0.225300,0.206398
450,0.210500,0.190168
600,0.189600,0.170790
750,0.155600,0.161390
900,0.158700,0.157821
1050,0.160600,0.143892
1200,0.156200,0.146462
1350,0.144300,0.139296
1500,0.127500,0.135379


🎉 Training finished!
✅ Saved model to: /content/drive/MyDrive/llama3_math_contest/SATversion1AM
==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 10000/10000 [49:06<00:00,  3.39it/s]

✅ submission.csv saved — upload to Kaggle!


The score for training and validation score are still dropping.
so increase the steps to see if this can get further.



# This is 11th submission

increase the prize to 300

increase the learning rate.

In [1]:
# =========================
# 0) Imports (Unsloth first)
# =========================
import unsloth
from unsloth import FastLanguageModel
from datasets import load_dataset, concatenate_datasets

import torch
from torch.nn.utils.rnn import pad_sequence
from transformers import TrainingArguments, EarlyStoppingCallback
from trl import SFTTrainer

import pandas as pd
from tqdm import tqdm

# --- Base config ---
max_seq_length = 2048
dtype = None          # auto bf16 on A100 / fp16 otherwise
load_in_4bit = True   # good for L4/A100

# --- Load base model once (for finetuning) ---
model_id = "unsloth/Meta-Llama-3.1-8B"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = model_id,
    max_seq_length = max_seq_length,
    dtype          = dtype,
    load_in_4bit   = load_in_4bit,
)
print("✅ Model and tokenizer loaded for training.")

# Ensure pad + left-truncation + left-padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"
tokenizer.padding_side    = "left"  # optional but consistent

# =========================================
# 1) LoRA
# =========================================
try:
    model = FastLanguageModel.get_peft_model(
        model,
        r = 16,
        lora_alpha = 32,
        target_modules = [
            "q_proj","k_proj","v_proj","o_proj",
            "gate_proj","up_proj","down_proj",
        ],
        lora_dropout = 0.05,
        bias = "none",
        use_gradient_checkpointing = "unsloth",
        random_state = 42,
    )
    print("✅ LoRA adapters configured.")
except Exception:
    print("⚠️ LoRA already present — continuing.")

# =========================================
# 2) Dataset formatting — balanced 20k train / 1k val
# =========================================
DATASET_ID = "ad6398/nyu-dl-teach-maths-comp"
full_dataset = load_dataset(DATASET_ID, split="train").shuffle(seed=42)

# Split by label (after global shuffle)
true_data  = full_dataset.filter(lambda x: x["is_correct"] == True)
false_data = full_dataset.filter(lambda x: x["is_correct"] == False)
print(len(true_data), len(false_data))

# Target sizes
train_per_class = 45_000   # 10k True + 10k False = 20k
val_per_class   = 2000      # 0.5k True + 0.5k False = 1k

train_true         = true_data.select(range(train_per_class))
train_false        = false_data.select(range(train_per_class))
validation_true    = true_data.select(range(train_per_class, train_per_class + val_per_class))
validation_false   = false_data.select(range(train_per_class, train_per_class + val_per_class))

# Merge + shuffle final sets
from datasets import concatenate_datasets
train_dataset       = concatenate_datasets([train_true, train_false]).shuffle(seed=42)
validation_dataset  = concatenate_datasets([validation_true, validation_false]).shuffle(seed=42)

print("✅ Final Balanced Dataset:")
print(f"Train size: {len(train_dataset)}   (True={train_per_class}, False={train_per_class})")
print(f"Validation size: {len(validation_dataset)} (True={val_per_class}, False={val_per_class})")

EOS_TOKEN      = tokenizer.eos_token or "<|end_of_text|>"
ANSWER_TAG     = "Your Final Answer (True/False):\n"
RESERVE_TOKENS = 12

PROMPT_TEMPLATE = (
    "You are a strict and highly accurate math verifier. And you are doing your best. Because you know you are getting $300 if you made a right verification\n You will loss $300 if you made a wrong verication!\n"
    "Your task:\n"
    "1) Recalculate the correct answer yourself.Make sure you consider everything so the solution and the answer you got it right! \n"
    "2) Compare it with the provided solution ONLY.\n"
    "3) Judge if the provided solution is correct.\n"
    "4) Step by step, make sure to check provided solution step by step.\n"
    "5) Becarful to the all small things!\n"
    "Important rules:\n"
    "- Think privately inside <scratch></scratch>.\n"
    "- DO NOT reveal intermediate calculations.\n"
    "- Output ONLY 'True' or 'False' after the final answer tag.\n\n"
    "Question:\n{question}\n\n"
    "<scratch>Step-by-step internal reasoning here (not visible to users)</scratch>\n"
    "Provided Solution:\n{provided_solution}\n\n"
    f"{ANSWER_TAG}"
)

def safe_join(question: str, provided_solution: str, label: str) -> str:
    prefix = PROMPT_TEMPLATE.format(question=question, provided_solution=provided_solution)
    ids = tokenizer(prefix, add_special_tokens=False)["input_ids"]
    if len(ids) > max_seq_length - RESERVE_TOKENS:
        extra = len(ids) - (max_seq_length - RESERVE_TOKENS)
        provided_solution = provided_solution[extra:]
        prefix = PROMPT_TEMPLATE.format(question=question, provided_solution=provided_solution)
    return prefix + label + EOS_TOKEN

def formatting_prompts_func(batch):
    texts = []
    for q, s, o in zip(batch["question"], batch["solution"], batch["is_correct"]):
        label = "True" if bool(o) else "False"
        texts.append(safe_join(q.strip(), str(s).strip(), label))
    return {"text": texts}

formatted_train_dataset      = train_dataset.map(formatting_prompts_func,      batched=True, num_proc=8)
formatted_validation_dataset = validation_dataset.map(formatting_prompts_func, batched=True, num_proc=8)

# Ensure the ANSWER_TAG exists (safety)
def _has_answer_tag(example):
    ids = tokenizer(example["text"], add_special_tokens=False)["input_ids"]
    tag_ids = tokenizer(ANSWER_TAG, add_special_tokens=False)["input_ids"]
    L, T = len(ids), len(tag_ids)
    for i in range(max(0, L - T + 1)):
        if ids[i:i+T] == tag_ids:
            return True
    return False

formatted_train_dataset      = formatted_train_dataset.filter(_has_answer_tag,      num_proc=8)
formatted_validation_dataset = formatted_validation_dataset.filter(_has_answer_tag, num_proc=8)

print("✅ Final Dataset:", len(formatted_train_dataset), len(formatted_validation_dataset))

# =========================================
# 3) Collator — completion-only after ANSWER_TAG
# =========================================
class CompletionOnlyCollator:
    def __init__(self, tokenizer, response_template, max_length=2048):
        self.tokenizer     = tokenizer
        self.max_length    = max_length
        self.template_ids  = tokenizer(response_template, add_special_tokens=False)["input_ids"]

    def _mask_after_template(self, input_ids: torch.Tensor) -> torch.Tensor:
        tpl    = torch.tensor(self.template_ids, dtype=torch.long)
        labels = input_ids.clone()
        L, T   = len(input_ids), len(tpl)
        start  = -1
        for i in range(max(0, L - T + 1)):
            if torch.equal(input_ids[i:i+T], tpl):
                start = i
                break
        if start == -1:
            labels[:] = -100
        else:
            labels[:start+T] = -100
        return labels

    def __call__(self, features):
        ids_list, attn_list, labels_list = [], [], []
        for f in features:
            enc   = self.tokenizer(f["text"], truncation=True, max_length=self.max_length, add_special_tokens=False)
            ids   = torch.tensor(enc["input_ids"], dtype=torch.long)
            attn  = torch.ones_like(ids, dtype=torch.long)
            labs  = self._mask_after_template(ids)
            ids_list.append(ids); attn_list.append(attn); labels_list.append(labs)
        input_ids      = pad_sequence(ids_list,  batch_first=True, padding_value=self.tokenizer.pad_token_id)
        attention_mask = pad_sequence(attn_list, batch_first=True, padding_value=0)
        labels         = pad_sequence(labels_list, batch_first=True, padding_value=-100)
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

collator = CompletionOnlyCollator(tokenizer, ANSWER_TAG, max_seq_length)

# =========================================
# 4) Training args
# =========================================
output_dir = "/content/drive/MyDrive/llama3_math_contest/SATversion6PM"

training_args = TrainingArguments(
    output_dir = output_dir,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 8,
    max_steps = 3000,
    warmup_steps = 150,
    learning_rate = 8e-5,
    lr_scheduler_type = "cosine",

    logging_steps     = 25,
    save_strategy     = "steps",
    save_steps        = 150,
    save_total_limit  = 2,

    eval_strategy           = "steps",
    eval_steps              = 150,
    load_best_model_at_end  = True,
    metric_for_best_model   = "eval_loss",
    greater_is_better       = False,

    remove_unused_columns = False,

    bf16  = torch.cuda.is_bf16_supported(),
    fp16  = not torch.cuda.is_bf16_supported(),
    optim = "adamw_8bit",
    weight_decay = 0.03,
    seed = 42,
    report_to = "none",
)

# =========================================
# 5) Train
# =========================================
trainer = SFTTrainer(
    model              = model,
    tokenizer          = tokenizer,
    train_dataset      = formatted_train_dataset,
    eval_dataset       = formatted_validation_dataset,
    dataset_text_field = "text",
    data_collator      = collator,
    max_seq_length     = max_seq_length,
    packing            = False,
    args               = training_args,
    callbacks          = [EarlyStoppingCallback(early_stopping_patience=3)],
)

print("✅ Begin training!")
trainer.train()
print("🎉 Training finished!")

# =========================================
# 6) Save
# =========================================
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print("✅ Saved model to:", output_dir)

# =========================================
# 7) Inference + submission
# =========================================
# Reload best checkpoint adapters
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = output_dir,
    max_seq_length = 2048,
    dtype          = None,
    load_in_4bit   = True,
)
model.eval().to("cuda")

def parse_true_false(text: str) -> bool:
    # robust to punctuation/newlines/spaces
    t = text.strip().lower()
    if t.startswith("true"):
        return True
    if t.startswith("false"):
        return False
    # fallback: first occurrence
    if "true" in t and "false" not in t:  return True
    if "false" in t and "true"  not in t: return False
    return False

test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

for ex in tqdm(test_dataset):
    q = ex["question"].strip()
    s = str(ex["solution"]).strip()

    prompt = PROMPT_TEMPLATE.format(question=q, provided_solution=s)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens = 8,     # small headroom
            do_sample      = False,
            temperature    = 0.0,
            eos_token_id   = tokenizer.eos_token_id,
            use_cache      = True,
        )

    # decode ONLY newly generated tokens (safer than splitting the full text)
    new_tokens = outputs[0, inputs["input_ids"].size(1):]
    decoded    = tokenizer.decode(new_tokens, skip_special_tokens=True)

    pred = parse_true_false(decoded)
    predictions.append(pred)

submission = pd.DataFrame({
    "ID": range(len(predictions)),
    "is_correct": predictions
})
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv saved — upload to Kaggle!")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

Step,Training Loss,Validation Loss
150,0.258300,0.237748
300,0.210700,0.188283
450,0.200700,0.174817
600,0.183100,0.156961
750,0.152900,0.155816
900,0.158700,0.157184
1050,0.156500,0.156956
1200,0.160400,0.147558
1350,0.146800,0.139516
1500,0.128600,0.135163


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


🎉 Training finished!
✅ Saved model to: /content/drive/MyDrive/llama3_math_contest/SATversion6PM
==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 10000/10000 [47:18<00:00,  3.52it/s]

✅ submission.csv saved — upload to Kaggle!
